In [1]:
import torch
import numpy as np
from IPython.display import Audio

from src.datasets.speech_dataset import SpeechDataset
from src.models.denoise_net import SpeechDenoiseNet

In [2]:
training_set = SpeechDataset(train=True)
validation_set = SpeechDataset(train=False)

  0%|          | 0/11572 [00:00<?, ?it/s]

Loading training dataset into memory.
Loading clean data...


  0%|          | 43/11572 [00:00<00:27, 419.30it/s]

Loading noisy data...


 79%|███████▉  | 650/824 [00:00<00:00, 6492.87it/s]

Done loading dataset into memory.
Loading validation dataset into memory.
Loading clean data...


 31%|███       | 256/824 [00:00<00:00, 2540.10it/s]

Loading noisy data...


100%|██████████| 824/824 [00:00<00:00, 961.62it/s] 

Done loading dataset into memory.


In [3]:
net = SpeechDenoiseNet().cpu()

In [4]:
CHECKPOINT_HIGH_LEVEL_FEATURE_LOSS = 'checkpoints/denoise-net-1570664133.ckpt' # Distored, but audible
CHECKPOINT_MORE_LAYERS_FEATURE_LOSS = 'checkpoints/denoise-net-1570667474.ckpt'  # Distored, but audible
CHECKPOINT_LOW_LEVEL_FEATURE_LOSS = 'checkpoints/denoise-net-1570668137.ckpt'  # Silence
ALL_LAYERS_FL = 'checkpoints/denoise-net-1570668767.ckpt'  # Distored, but audible
WEIGHTER_LAYERS = 'checkpoints/denoise-net-1570669439.ckpt'  # Distored, but audible
net.load_state_dict(torch.load(ALL_LAYERS_FL))

<All keys matched successfully>

In [5]:
idx = 1
in_wav_arr = training_set[idx][0].numpy()
clean_wav_arr = training_set[idx][1].numpy()
inputs = torch.tensor(in_wav_arr).float().cpu()
inputs = inputs.view(1, 1, -1)
outputs = net(inputs)
outputs = outputs.squeeze(dim=0)
wav_arr = outputs.cpu().detach().numpy()
print('Predicted clean signal')
Audio(wav_arr, rate=16000)

Predicted clean signal


In [6]:
print('Noisy signal')
Audio(in_wav_arr, rate=16000)

Noisy signal


In [7]:
print('Actual clean signal')
Audio(clean_wav_arr, rate=16000)

Actual clean signal
